In [1]:
from google.colab import drive
drive.mount('../content/drive')

%matplotlib inline
import cv2
import numpy as np
import matplotlib.pyplot as plt
import copy
from google.colab.patches import cv2_imshow
import glob
import multiprocess as mp
import re
import time

Mounted at ../content/drive


In [ ]:
#vidualWordを使った予測.multiprocess使わないVer

#処理時間計測用
start = time.time()

num_words = 128
#学習用画像を格納しているfileをすべて指定
filepaths =glob.glob("/content/drive/MyDrive/Colab Notebooks/lec13/data/train/*")
filepaths_cheetah = list() #cheetahのファイル名list
filepaths_leopard = list() #leopardのファイル名list
for fname in filepaths:
  if re.search("2\d{5}\.(jpg|jpeg)$", fname):
    filepaths_cheetah.append(fname)
  elif re.search("1\d{5}\.(jpg|jpeg)$", fname):
    filepaths_leopard.append(fname)

#インスタンス作成
kaze = cv2.KAZE_create()
#インスタンス作成
bowTrainer = cv2.BOWKMeansTrainer(num_words)

grayImg_kp_list_cheetah = list()
grayImg_kp_list_leopard = list()

files_count = len(filepaths)
progress_count = 0
for fname in filepaths_cheetah:
  progress_count += 1
  if progress_count % 100 == 0:
    print("progress: {}/{}".format(progress_count, files_count))
  img = cv2.imread(fname)
  #画像をグレー画像に
  gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  #局所特徴点、局所特徴量の取得
  kp, descriptor = kaze.detectAndCompute(gray_img, None)
  grayImg_kp_list_cheetah.append([gray_img, kp])
  bowTrainer.add(descriptor)

for fname in filepaths_leopard:
  progress_count += 1
  if progress_count % 100 == 0:
    print("progress: {}/{}".format(progress_count, files_count))
  img = cv2.imread(fname)
  #画像をグレー画像に
  gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  #局所特徴点、局所特徴量の取得
  kp, descriptor = kaze.detectAndCompute(gray_img, None)
  grayImg_kp_list_leopard.append([gray_img, kp])
  bowTrainer.add(descriptor)
print("progress: {}/{}".format(progress_count, files_count))

  
#Visual Wordsの決定
#特徴量をk-means法でクラスタリング
codebook = bowTrainer.cluster()
#画像毎のBoVMベクトルの作成

bf = cv2.BFMatcher() #引数無し
#インスタンス作成
bowExtractor = cv2.BOWImgDescriptorExtractor(kaze, bf)
#bowExtractorにcodebookをセット
bowExtractor.setVocabulary(codebook)

cheetah_list_len = len(filepaths_cheetah)
leopard_list_len = len(filepaths_leopard)
#ベクトルを格納する配列。theetahとleopardに分けておき、後で連結する
all_bowDescriptors_theetah = np.empty((cheetah_list_len, num_words), dtype=np.float32)
all_bowDescriptors_leopard = np.empty((leopard_list_len, num_words), dtype=np.float32)
#ラベルを格納する配列。theetahとleopardに分けておき、後で連結する
all_labels_theetah = np.empty((cheetah_list_len, 1), dtype=np.int64)
all_labels_leopard = np.empty((leopard_list_len, 1), dtype=np.int64)

#cheetah
for i, gray_img_and_kp in enumerate(grayImg_kp_list_cheetah):
  #入力画像のBoVMベクトル作成
  bowDescriptor = bowExtractor.compute(gray_img_and_kp[0], gray_img_and_kp[1])
  #結果を格納
  all_bowDescriptors_theetah[i] = bowDescriptor
  all_labels_theetah[i][0] = 0
#leopard
for i, gray_img_and_kp in enumerate(grayImg_kp_list_leopard):
  #入力画像のBoVMベクトル作成
  bowDescriptor = bowExtractor.compute(gray_img_and_kp[0], gray_img_and_kp[1])
  #結果を格納
  all_bowDescriptors_leopard[i] = bowDescriptor
  all_labels_leopard[i][0] = 1

all_bowDescriptors = np.concatenate([all_bowDescriptors_theetah, all_bowDescriptors_leopard])
all_labels = np.concatenate([all_labels_theetah, all_labels_leopard])
#インスタンス作成
svm = cv2.ml.SVM_create()
#SVMの設定。直線の判別線を得る
svm.setType(cv2.ml.SVM_C_SVC)
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setGamma(1)
svm.setC(1)
svm.setTermCriteria((cv2.TERM_CRITERIA_COUNT, 100, 1.e-06))
#SVMの学習
svm.train(all_bowDescriptors, cv2.ml.ROW_SAMPLE, all_labels)
#新しい画像に対してラベルを予測

#test用画像を格納しているfileをすべて指定
filepaths_test =glob.glob("/content/drive/MyDrive/Colab Notebooks/lec13/data/test/*")
filepaths_cheetah_test = list() #cheetahのファイル名list
filepaths_leopard_test = list() #leopardのファイル名list
for fname in filepaths_test:
  if re.search("2\d{5}\.(jpg|jpeg)$", fname):
    filepaths_cheetah_test.append(fname)
  elif re.search("1\d{5}\.(jpg|jpeg)$", fname):
    filepaths_leopard_test.append(fname)

num_test = len(filepaths_test) # test枚数
corrects_num = 0 #正解した数

#test画像をひとつずつ予測
for file_path in filepaths_cheetah_test:
  test_img = cv2.imread(file_path)
  #画像をグレー画像に
  gray_test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
  #局所特徴点、局所特徴量の取得
  kp, descriptor = kaze.detectAndCompute(gray_test_img, None)
  #test画像のBoVMベクトル作成
  bowDescriptors = bowExtractor.compute(gray_test_img, kp)
  _, predict = svm.predict(bowDescriptors)
  predict = int(predict[0][0])
  if predict == 0:
    corrects_num += 1
for file_path in filepaths_leopard_test:
  test_img = cv2.imread(file_path)
  #画像をグレー画像に
  gray_test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
  #局所特徴点、局所特徴量の取得
  kp, descriptor = kaze.detectAndCompute(gray_test_img, None)
  #test画像のBoVMベクトル作成
  bowDescriptors = bowExtractor.compute(gray_test_img, kp)
  _, predict = svm.predict(bowDescriptors)
  predict = int(predict[0][0])
  if predict == 1:
    corrects_num += 1
    
#正答率
print("正答率は{}でした({}/{})".format(corrects_num/num_test, corrects_num, num_test))

#処理時間
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

progress: 100/895
progress: 200/895
progress: 300/895
progress: 400/895
progress: 500/895
progress: 600/895
progress: 700/895
progress: 800/895
progress: 895/895
正答率は0.7333333333333333でした(22/30)
elapsed_time:1371.4868597984314[sec]


In [ ]:
#vidualWordを使った予測.multiprocessで早くならないか試してみた
#逆に遅くなったので、上のプログラムを使った方が良い

#処理時間計測用
start = time.time()

num_words = 128
#学習用画像を格納しているfileをすべて指定
filepaths =glob.glob("/content/drive/MyDrive/Colab Notebooks/lec13/data/train/*")
filepaths_cheetah = list() #cheetahのファイル名list
filepaths_leopard = list() #leopardのファイル名list
for fname in filepaths:
  if re.search("2\d{5}\.(jpg|jpeg)$", fname):
    filepaths_cheetah.append(fname)
  elif re.search("1\d{5}\.(jpg|jpeg)$", fname):
    filepaths_leopard.append(fname)

#インスタンス作成
kaze = cv2.KAZE_create()
#インスタンス作成
bowTrainer = cv2.BOWKMeansTrainer(num_words)
fname_kp_list_theetah = list()
fname_kp_list_leopard = list()

with mp.Manager() as manager:
  # マネージャーからValueクラスを作成
  files_count = manager.Value('i', len(filepaths))
  progress_count = manager.Value('i', 0)
  fname_kp_list_theetah_manager = manager.list()
  fname_kp_list_leopard_manager = manager.list()

  def descriptorCreateTheetah(fname):
    progress_count.value += 1
    if progress_count.value % 100 == 0:
      print("progress: {}/{}".format(progress_count.value, files_count.value))
    img = cv2.imread(fname)
    #画像をグレー画像に
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #局所特徴点、局所特徴量の取得
    kp, descriptor = kaze.detectAndCompute(gray_img, None)
    #kpをpickleできるようにする
    index = []
    for p in kp:
      temp = (p.pt, p.size, p.angle, p.response, p.octave, p.class_id)
      index.append(temp)
    fname_kp_list_theetah_manager.append([fname, index])
    #特徴量をreturn
    return(descriptor)

  def descriptorCreateLeopard(fname):
    progress_count.value += 1
    if progress_count.value % 100 == 0:
      print("progress: {}/{}".format(progress_count.value, files_count.value))
    img = cv2.imread(fname)
    #画像をグレー画像に
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #局所特徴点、局所特徴量の取得
    kp, descriptor = kaze.detectAndCompute(gray_img, None)
    #kpをpickleできるようにする
    index = []
    for p in kp:
      temp = (p.pt, p.size, p.angle, p.response, p.octave, p.class_id)
      index.append(temp)
    fname_kp_list_leopard_manager.append([fname, index])
    #特徴量をreturn
    return(descriptor)

  with mp.Pool() as p:
    descriptors_cheetah = p.map(descriptorCreateTheetah, filepaths_cheetah)
    descriptors_leopard = p.map(descriptorCreateLeopard, filepaths_leopard)
    fname_kp_list_theetah = fname_kp_list_theetah_manager[:]
    fname_kp_list_leopard = fname_kp_list_leopard_manager[:]
    print("progress: {}/{}".format(progress_count.value, files_count.value))

for des in descriptors_cheetah:
  bowTrainer.add(des)
for des in descriptors_leopard:
  bowTrainer.add(des)
  
#Visual Wordsの決定
#特徴量をk-means法でクラスタリング
codebook = bowTrainer.cluster()
#画像毎のBoVMベクトルの作成

bf = cv2.BFMatcher() #引数無し
#インスタンス作成
bowExtractor = cv2.BOWImgDescriptorExtractor(kaze, bf)
#bowExtractorにcodebookをセット
bowExtractor.setVocabulary(codebook)

cheetah_list_len = len(filepaths_cheetah)
leopard_list_len = len(filepaths_leopard)
#ベクトルを格納する配列。theetahとleopardに分けておき、後で連結する
all_bowDescriptors_cheetah = np.empty((cheetah_list_len, num_words), dtype=np.float32)
all_bowDescriptors_leopard = np.empty((leopard_list_len, num_words), dtype=np.float32)
#ラベルを格納する配列。theetahとleopardに分けておき、後で連結する
all_labels_cheetah = np.empty((cheetah_list_len, 1), dtype=np.int64)
all_labels_leopard = np.empty((leopard_list_len, 1), dtype=np.int64)

def bowDescriptorCreate(list):
  fname = list[0]
  img = cv2.imread(fname)
  gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  kp = []
  for p in list[1]:
    temp = cv2.KeyPoint(x=p[0][0], y=p[0][1], _size=p[1], _angle=p[2],
                        _response=p[3], _octave=p[4], _class_id=p[5])
    kp.append(temp)
  #入力画像のBoVMベクトル作成
  bowDescriptor = bowExtractor.compute(gray_img, kp)
  return(bowDescriptor)

mp.set_start_method('fork', force=True)
with mp.Pool() as p:
  bowDescriptors_cheetah = p.map(bowDescriptorCreate, fname_kp_list_theetah)
  bowDescriptors_leopard = p.map(bowDescriptorCreate, fname_kp_list_leopard)
#cheetah
for i, bowDes in enumerate(bowDescriptors_cheetah):
  #結果を格納
  all_bowDescriptors_cheetah[i] = bowDes
  all_labels_cheetah[i][0] = 0

#leopard
for i, bowDes in enumerate(bowDescriptors_leopard):
  #結果を格納
  all_bowDescriptors_leopard[i] = bowDes
  all_labels_leopard[i][0] = 1

all_bowDescriptors = np.concatenate([all_bowDescriptors_cheetah, all_bowDescriptors_leopard])
all_labels = np.concatenate([all_labels_cheetah, all_labels_leopard])
#インスタンス作成
svm = cv2.ml.SVM_create()
#SVMの設定。直線の判別線を得る
svm.setType(cv2.ml.SVM_C_SVC)
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setGamma(1)
svm.setC(1)
svm.setTermCriteria((cv2.TERM_CRITERIA_COUNT, 100, 1.e-06))
#SVMの学習
svm.train(all_bowDescriptors, cv2.ml.ROW_SAMPLE, all_labels)
#新しい画像に対してラベルを予測

#test用画像を格納しているfileをすべて指定
filepaths_test =glob.glob("/content/drive/MyDrive/Colab Notebooks/lec13/data/test/*")
filepaths_cheetah_test = list() #cheetahのファイル名list
filepaths_leopard_test = list() #leopardのファイル名list
for fname in filepaths_test:
  if re.search("2\d{5}\.(jpg|jpeg)$", fname):
    filepaths_cheetah_test.append(fname)
  elif re.search("1\d{5}\.(jpg|jpeg)$", fname):
    filepaths_leopard_test.append(fname)

num_test = len(filepaths_test) # test枚数
corrects_num = 0 #正解した数

#test画像をひとつずつ予測
for file_path in filepaths_cheetah_test:
  test_img = cv2.imread(file_path)
  #画像をグレー画像に
  gray_test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
  #局所特徴点、局所特徴量の取得
  kp, descriptor = kaze.detectAndCompute(gray_test_img, None)
  #test画像のBoVMベクトル作成
  bowDescriptors = bowExtractor.compute(gray_test_img, kp)
  _, predict = svm.predict(bowDescriptors)
  predict = int(predict[0][0])
  if predict == 0:
    corrects_num += 1
for file_path in filepaths_leopard_test:
  test_img = cv2.imread(file_path)
  #画像をグレー画像に
  gray_test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
  #局所特徴点、局所特徴量の取得
  kp, descriptor = kaze.detectAndCompute(gray_test_img, None)
  #test画像のBoVMベクトル作成
  bowDescriptors = bowExtractor.compute(gray_test_img, kp)
  _, predict = svm.predict(bowDescriptors)
  predict = int(predict[0][0])
  if predict == 1:
    corrects_num += 1
    
#正答率
print("正答率は{}でした({}/{})".format(corrects_num/num_test, corrects_num, num_test))

#処理時間
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

progress: 100/895
progress: 200/895
progress: 300/895
progress: 400/895
progress: 500/895
progress: 600/895
progress: 700/895
progress: 800/895
progress: 890/895
正答率は0.7でした(21/30)
elapsed_time:1432.8737156391144[sec]


In [ ]:
#マルチプロセスの復習
import multiprocessing as mp

cpu_num = mp.cpu_count()
print(cpu_num)

# 並列処理させる関数
def nijou(x):
    #処理
    return(x+1)

p = mp.Pool(cpu_num) # プロセス数を設定
result = p.map(nijou, [1, 2, 3, 4, 5])
print(result)

2
[2, 3, 4, 5, 6]


In [ ]:
files =glob.glob("/content/drive/MyDrive/Colab Notebooks/lec13/hyou/*")

for i, fname in enumerate(files):
    n = i*4
    name_list = [str(n) + "1.jpg" for n in range(4)]
    img = cv2.imread(fname)
    cv2.imwrite("/content/drive/MyDrive/Colab Notebooks/lec13/hyouOut/" + name_list[0], img)
    img1 = cv2.flip(img, 1)
    cv2.imwrite("/content/drive/MyDrive/Colab Notebooks/lec13/hyouOut/" + name_list[1], img1)
    w, h, _ = img.shape
    img2 = img[1:w, 1:h]
    cv2.imwrite("/content/drive/MyDrive/Colab Notebooks/lec13/hyouOut/" + name_list[2], img2)
    img3 = cv2.resize(img, dsize=None, fx=1.0, fy=1.05)
    cv2.imwrite("/content/drive/MyDrive/Colab Notebooks/lec13/hyouOut/" + name_list[3], img3)
